In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
from tqdm import tqdm
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [ ]:
! pip install gdown

In [ ]:
! gdown https://drive.google.com/uc?id=1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo

Downloading...
From: https://drive.google.com/uc?id=1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo
To: /content/vgg_face_weights.h5
580MB [00:10, 55.4MB/s]


In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
model.load_weights('vgg_face_weights.h5')

In [ ]:
vgg_face_feature1=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)
vgg_face_feature2=Model(inputs=model.layers[0].input,outputs=model.layers[-5].output)
vgg_face_feature3=Model(inputs=model.layers[0].input,outputs=model.layers[-7].output)

In [ ]:
vgg_face_feature1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0     

In [ ]:
train_dirs = glob.glob("drive/MyDrive/images1/*")
train_dirs.sort()
num_classes = 82
train_dirs = train_dirs[:num_classes]
print(train_dirs)
data_fc3 =[] 
data_fc2 = []
data_fc1 = []
id_person = []
for train_dir in tqdm(train_dirs):
  imgPaths = glob.glob(train_dir + "/*.JPG")
  imgPaths.sort()
  for imgPath in tqdm(imgPaths):
    image = load_img(imgPath, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image , axis=0)
    image=preprocess_input(image)
    image_encode_fc3=vgg_face_feature1(image)
    image_encode_fc2=vgg_face_feature2(image)
    image_encode_fc1=vgg_face_feature3(image)
    data_fc3.append(np.squeeze(K.eval(image_encode_fc3)).tolist())
    data_fc2.append(np.squeeze(K.eval(image_encode_fc2)).tolist())
    data_fc1.append(np.squeeze(K.eval(image_encode_fc1)).tolist())

    id_label = imgPath.split(os.path.sep)[-2]
    id_label = int(id_label)-1
    id_person.append(id_label) 

print("id:",id_person)

['drive/MyDrive/images1/001', 'drive/MyDrive/images1/002', 'drive/MyDrive/images1/003', 'drive/MyDrive/images1/004', 'drive/MyDrive/images1/005', 'drive/MyDrive/images1/006', 'drive/MyDrive/images1/007', 'drive/MyDrive/images1/008', 'drive/MyDrive/images1/009', 'drive/MyDrive/images1/010', 'drive/MyDrive/images1/011', 'drive/MyDrive/images1/012', 'drive/MyDrive/images1/013', 'drive/MyDrive/images1/014', 'drive/MyDrive/images1/015', 'drive/MyDrive/images1/016', 'drive/MyDrive/images1/017', 'drive/MyDrive/images1/018', 'drive/MyDrive/images1/019', 'drive/MyDrive/images1/020', 'drive/MyDrive/images1/021', 'drive/MyDrive/images1/022', 'drive/MyDrive/images1/023', 'drive/MyDrive/images1/024', 'drive/MyDrive/images1/025', 'drive/MyDrive/images1/026', 'drive/MyDrive/images1/027', 'drive/MyDrive/images1/028', 'drive/MyDrive/images1/029', 'drive/MyDrive/images1/030', 'drive/MyDrive/images1/031', 'drive/MyDrive/images1/032', 'drive/MyDrive/images1/033', 'drive/MyDrive/images1/034', 'drive/MyDriv

 20%|██        | 3/15 [00:06<00:24,  2.06s/it]

100%|██████████| 82/82 [32:06<00:00, 23.49s/it]

id: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 

In [ ]:
print(len(data_fc1),len(data_fc2),len(data_fc3),len(id_person))
data_fc1 = np.array(data_fc1)
data_fc2 = np.array(data_fc2)
data_fc3 = np.array(data_fc3)
id_person = np.array(id_person)
print(data_fc1[0].shape," ",data_fc2[0].shape," ", data_fc3[0].shape)
print(data_fc1.shape," ",data_fc2.shape," ", data_fc3.shape)

920 920 920 920
(4096,)   (4096,)   (2622,)
(920, 4096)   (920, 4096)   (920, 2622)


In [ ]:
data_comb_fc1_fc2 = np.hstack((data_fc1 , data_fc2))
data_comb_fc1_fc3 = np.hstack((data_fc1 , data_fc3))
data_comb_fc2_fc3 = np.hstack((data_fc2 , data_fc3))
data_comb_fc1_fc2_fc3 = np.hstack((data_fc1 , data_fc2 , data_fc3))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
tuned_parameters =  [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [0.001, 0.01, 0.1, 10, 25, 50, 100]}]
clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring ='accuracy' , verbose = 3)
clf.fit(data_fc1,id_person)


Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.6s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.6s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   8.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.1s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   8.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.5s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   8.5s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   8.5s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   8.5s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   8.6s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 19.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.775


In [ ]:
clf.fit(data_fc2,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.8s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   8.8s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.6s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   8.8s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.7s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   8.7s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   8.3s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   8.2s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   8.2s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   8.2s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 20.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.8130434782608695


In [ ]:
clf.fit(data_fc3,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   5.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.5s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   5.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.0s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=   5.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   5.5s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=   5.6s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   5.3s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   5.3s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=   5.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=   5.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 12.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.7804347826086956


In [ ]:
clf.fit(data_comb_fc1_fc2,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  17.1s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.2s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  17.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   34.2s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  17.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  17.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  17.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  17.1s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  17.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  17.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  17.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 39.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.7630434782608695


In [ ]:
clf.fit(data_comb_fc1_fc3,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  14.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.0s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  13.9s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   27.9s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  13.9s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  13.9s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  13.9s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  14.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  13.9s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  13.9s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  13.9s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 32.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.7717391304347826


In [ ]:
clf.fit(data_comb_fc2_fc3,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  14.2s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.2s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  14.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   28.2s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  14.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  14.0s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  14.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  13.0s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  13.1s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  13.1s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  13.1s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 32.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.8152173913043479


In [ ]:
clf.fit(data_comb_fc1_fc2_fc3,id_person)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  22.7s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.7s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  22.4s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   45.2s remaining:    0.0s


[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.022, total=  22.4s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  22.4s
[CV] C=0.001, gamma=0.01, kernel=rbf .................................
[CV] ..... C=0.001, gamma=0.01, kernel=rbf, score=0.016, total=  22.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  22.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  22.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.022, total=  22.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .... C=0.001, gamma=0.001, kernel=rbf, score=0.016, total=  22.4s
[CV] C=0.001, gamma=0.001, kernel=rbf ................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed: 52.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.001, 0.01, 0.1, 10, 25, 50, 100],
                          'gamma': [0.01, 0.001, 0.0001, 1e-05],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.7532608695652174
